In [1]:
import nltk

In [2]:
nltk.download('twitter_samples')
from nltk.corpus import twitter_samples

[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


In [3]:
#Preprocessing - Tokenization
#Punkt is a pre-trained model that helps you tokenize words and sentences.
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
#The json files are datasets of tweets that are labelled as either positive, negative, or neutral.
positive_tweets = twitter_samples.strings('positive_tweets.json') # has 5000
#tweets with positive sentiments
negative_tweets = twitter_samples.strings('negative_tweets.json') # has 5000
#tweets with negative sentiments
text = twitter_samples.strings('tweets.20150430-223406.json') # has 20000 tweets
#with neutral/no sentiments

In [5]:
# tokenize a tweet from positive_tweets.json using the tokenized function and printing it
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
print(tweet_tokens[0])

['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'being', 'top', 'engaged', 'members', 'in', 'my', 'community', 'this', 'week', ':)']


In [6]:
#Preprocessing - Normalization
'''The wordnet resource is a lexical database for the English language that helps the script determine the
base word, while the averaged_perceptron_tagger resource helps determine the context of a word in a
sentence'''
# downloading both resources
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [20]:
 
from nltk.tag import pos_tag
from nltk.corpus import twitter_samples
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
print(pos_tag(tweet_tokens[0])) # POS-tags the tokens in the first tweet in the positive_tweets.json dataset

[('#FollowFriday', 'JJ'), ('@France_Inte', 'NNP'), ('@PKuchly57', 'NNP'), ('@Milipol_Paris', 'NNP'), ('for', 'IN'), ('being', 'VBG'), ('top', 'JJ'), ('engaged', 'VBN'), ('members', 'NNS'), ('in', 'IN'), ('my', 'PRP$'), ('community', 'NN'), ('this', 'DT'), ('week', 'NN'), (':)', 'NN')]


In [21]:
'''This code imports the WordNetLemmatizer class and initializes it to a variable, lemmatizer.
The function lemmatize_sentence first gets the position tag of each token of a tweet. Within the if
statement, if the tag starts with NN, the token is assigned as a noun. Similarly, if the tag starts with VB, the
token is assigned as a verb'''

from nltk.stem.wordnet import WordNetLemmatizer
def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'): # we identify tokens tagged with "NN" as nouns
            pos = 'n'
        elif tag.startswith('VB'): # we identify tokens tagged with "VB" as verbs
            pos = 'v'
        else:
            pos = 'a'
# we are getting the "lemma" of each token with the token and their POS as arguments and adding the resulting
# lemma to the lemmatized_sentence array
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
        return lemmatized_sentence
print(lemmatize_sentence(tweet_tokens[0])) # this prints the tokens as their root form

['#FollowFriday']


In [22]:
#Preprocessing - Removing Noise
'''Noise is any part of the text that does not add meaning or information to data. For instance, the most
common words in a language are called stop words. Some examples of stop words are “is”, “the”, and “a”.'''

'''The modules re and string are also imported to use in the noise-removal process. The re module stands
for regular expressions and is used for pattern matching and manipulation of strings, while the string
module provides a collection of string constants and functions to work with strings.'''

import re, string
def remove_noise(tweet_tokens, stop_words=()):
    cleaned_tokens = []
    
    for token, tag in pos_tag(tweet_tokens):
        # removes all URLs or anything that starts with http:// or https://
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                      '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        # removes all username mentions or anything that starts with @
        token = re.sub("(@[A-Za-z0-9_]+)","", token)
        
        # we are adding the lemmatize_sentence function from earlier here
        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
            
        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)
        # this line adds all tokens with more than zero length that are not punctuation marks and
        # are not stop words in the cleaned_tokens array
        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens


In [23]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english') # specifying to use only stop words in the English language
print(remove_noise(tweet_tokens[0], stop_words)) # we are using the remove_noise function defined in the previous cell

['#followfriday', 'top', 'engage', 'member', 'community', 'week', ':)']


In [25]:
# Comparing Tokens Before and After Preprocessing
positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))
for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

In [26]:
print("This are the tokens before preprocessing: ",positive_tweet_tokens[500],"\n")
print("This are the tokens after preprocessing:",positive_cleaned_tokens_list[500])

This are the tokens before preprocessing:  ['Dang', 'that', 'is', 'some', 'rad', '@AbzuGame', '#fanart', '!', ':D', 'https://t.co/bI8k8tb9ht'] 

This are the tokens after preprocessing: ['dang', 'rad', '#fanart', ':d']


In [27]:
#Determining Word Density
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token
all_pos_words = get_all_words(positive_cleaned_tokens_list)

In [28]:
from nltk import FreqDist

freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))

[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]


In [29]:
#Preparing Data for the Model
'''converts the tweets from a list of cleaned tokens to dictionaries with keys as the tokens and True
as values. The corresponding dictionaries are stored in positive_tokens_for_model and.
negative_tokens_for_model'''

def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)
positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

In [30]:
#Splitting the Dataset for Training and Testing the Model
import random
# attaches the positive or negative label
positive_dataset = [(tweet_dict, "Positive")
                     for tweet_dict in positive_tokens_for_model]
negative_dataset = [(tweet_dict, "Negative")
                    for tweet_dict in negative_tokens_for_model]

# putting the positive and negative dataset into one combined dataset
dataset = positive_dataset + negative_dataset
# shuffling the dataset so sentiments are scattered throughout the dataset
random.shuffle(dataset)
# assigning the train and test data; train is from 0 to 6999 and test is 7000 to 9999
train_data = dataset[:7000]
test_data = dataset[7000:]

In [31]:
# Building and Testing the Model
#use the NaiveBayesClassifier class to build the model
from nltk import classify
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_data)
print("Accuracy is:", classify.accuracy(classifier, test_data))
print(classifier.show_most_informative_features(10))

Accuracy is: 0.9956666666666667
Most Informative Features
                      :( = True           Negati : Positi =   2065.2 : 1.0
                      :) = True           Positi : Negati =   1636.3 : 1.0
                     sad = True           Negati : Positi =     35.2 : 1.0
                follower = True           Positi : Negati =     22.7 : 1.0
                    luck = True           Positi : Negati =     15.8 : 1.0
                     x15 = True           Negati : Positi =     15.6 : 1.0
                    damn = True           Negati : Positi =     14.9 : 1.0
                  arrive = True           Positi : Negati =     14.2 : 1.0
              appreciate = True           Positi : Negati =     13.1 : 1.0
                   thank = True           Positi : Negati =     12.6 : 1.0
None


In [1]:
from nltk.tokenize import word_tokenize
#custom_tweet = "I ordered just once from TerribleCo, they screwed up, never used the app again."
#custom_tweet = 'Congrats #SportStar on your 7th best goal from last season winning goal of the year :) #Baller #Topbin #oneofmanyworldies'
custom_tweet = 'The plane ride was not very smooth'
custom_tokens = remove_noise(word_tokenize(custom_tweet))
print(classifier.classify(dict([token, True] for token in custom_tokens)))

NameError: name 'remove_noise' is not defined